In [2]:
import obspy
from datetime import datetime
import obspy.signal
import obspy.signal.filter
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('ggplot')

catalog = pd.read_csv('./data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv')
data_path = './data/lunar/training/data/S12_GradeA/'

In [ ]:
import os
import glob
import random
import numpy as np

N_TRACES = 40

trace_files = glob.glob("./data/lunar/training/data/S12_GradeA/*.mseed")[:40]
traces = [obspy.read(f)[0].detrend().filter("highpass", freq=1, corners=1).decimate(4) for f in trace_files]

catalog = pd.read_csv('./data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv')
arrival_rel_times = []
for i, quake in zip(range(N_TRACES), catalog.iloc):
    arrival_rel_times.append((datetime.strptime(quake['time_abs(%Y-%m-%dT%H:%M:%S.%f)'], '%Y-%m-%dT%H:%M:%S.%f') - traces[i].stats.starttime.datetime).total_seconds())

print(f"loaded {len(trace_files)} traces")


In [ ]:
from keras.layers import Input, Dense, LSTM
from keras.layers import Conv1D, MaxPooling1D, UpSampling1D
from keras.layers import BatchNormalization
from keras.layers import Dropout, Activation, Flatten
from keras.models import Model
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

num_traces = len(traces)

TT_SPLIT = 0.2

scaler = MinMaxScaler(feature_range=(-1,1))
trace = [trace.data for trace in traces]

x_train, x_test, y_train, y_test = train_test_split(trace, arrival_rel_times, test_size=TT_SPLIT, random_state=2)
max_len = max(len(max(x_train, key=len)), len(max(x_test, key=len)))
x_train = np.array(pad_sequences(x_train, maxlen=max_len)).astype(np.float32)
scaler.fit_transform(x_train)
x_train = x_train[..., None]
y_train = np.array(y_train).astype(np.float32)
x_test = np.array(pad_sequences(x_test, maxlen=max_len)).astype(np.float32)
scaler.transform(x_test)
x_test = x_test[..., None]
y_test = np.array(y_test).astype(np.float32)

# they are different lengths for some reason
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from tqdm.keras import TqdmCallback

batch_size = 8
epochs = 100
filters = [32, 64, 64]
kernel_sizes = [4, 8, 32]
pool_size = 2

hidden_act = 'leaky_relu'
final_act = 'linear'
optimizer = 'adam'
loss = 'huber'
padding = 'same'

input = Input(shape=(x_train.shape[1:]))

x = Conv1D(filters=filters[0], kernel_size=kernel_sizes[0], activation=hidden_act, padding=padding)(input)
x = MaxPooling1D(pool_size=pool_size, padding=padding)(x)
x = Conv1D(filters=filters[1], kernel_size=kernel_sizes[1], activation=hidden_act, padding=padding)(x)
x = MaxPooling1D(pool_size=pool_size, padding=padding)(x)
x = Conv1D(filters=filters[2], kernel_size=kernel_sizes[2], activation=hidden_act, padding=padding)(x)
x = MaxPooling1D(pool_size=pool_size, padding=padding)(x)

cnn_features = Flatten()(x)

# x = LSTM

x = Dense(units=16, activation=hidden_act)(cnn_features)
x = BatchNormalization()(x)
x = Dense(units=8, activation=hidden_act)(x)
x = BatchNormalization()(x)
dense = Dense(units=1, activation=final_act)(x)

model = Model(input, dense, name="pe23123")
model.compile(optimizer=optimizer, loss=loss)
model.summary()

hist = model.fit(x=x_train, y=y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test), callbacks=[TqdmCallback(epochs)])

train_loss = hist.history['loss']
last_train_loss_value = train_loss[len(train_loss)-1]
val_loss = hist.history['val_loss']
last_val_loss_value = val_loss[len(val_loss) - 1]

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
ax.plot(hist.history['loss'])
ax.plot(hist.history['val_loss'])
ax.set_title('Model Loss')
ax.set_ylabel('loss')
ax.set_xlabel('epoch')
ax.legend(['Train', 'Validation'], loc='upper left')
plt.savefig("./models", bbox_inches='tight')

model_json = model.to_json()
with open(f"./models/model-{datetime.datetime.now().time()}", "w") as json_file:
    json_file.write(model_json)

model.save_weights(f"./models/modelweights-{datetime.datetime.now().time()}")